# Assignment 8

### 1.复习课上内容， 阅读相应论文。

### 2. 回答以下理论题目

#### 1.  What is autoencoder?

自编码器是一种非监督学习的神经网络，encoder部分可以有效的对输入数据压缩降维，然后提取数据中的feature map，decoder负责将feature map中数据恢复，计算恢复的数据与原数据的误差并进行反向传递，逐步提高自编码的准确性。训练完成后自编码器即可提取出原数据的精髓（feature map），然后在后续训练学习中只学习此feature map即可，减少后续神经网络的训练负担。

#### 2. What are the differences between greedy search and beam search?

生成第一个词的分布之后，greedy search会根据条件语言模型选出概率最大的第一个词，之后每个时间点生成词分布后都会找出概率最大的词，而beam search每一步会选出概率最大的前N个词，再根据这N个词分别评估第二个词的概率，之后用第一个词的概率与第二个词的概率相乘，得到第一第二单词对的概率，选出前N个，以此类推，计算的是一整句话出现的最大概率。
相比greedy search，beam search考虑了更多的候选搜索空间，可得到更多的句子生成结果。

#### 3. What is the intuition of attention mechanism?

注意力机制会对输入的数据计算注意力权重来判断那些词重要，让神经网络只注意一部分数据，对长句子翻译有较好表现。

#### 4. What is the disadvantage of word embeding introduced in previous lectures ?

word embedding没办法解决一词多义问题，因为每个词只有一个固定的词向量。

#### 5. What is the architecture of ELMo model. (A brief description is enough)

ELMo是一个双层双向的LSTM语言模型，把每一层的隐状态包括第一层的词向量进行权重上的加和。先在一个大型的预料库上面进行训练，用模型的内部状态来表征一个词的向量，此时内部状态混合了所有的语义；然后将下游任务的文本输入模型，此时的词具备了具体的上下文语境，再用内部状态的线性组合来表征词，这种线性组合是在任务模型上进行训练得到的。

#### 6. Compared to RNN,  what is the advantage of Transformer ?

Transformer 中最重要的就是自注意力机制，这种在序列内部执行 Attention 的方法可以视为搜索序列内部的隐藏关系，这种内部关系对于翻译以及序列任务的性能有显著提升。比如遇到代词"it"时，transformer中自注意力机制层会根据其他语义信息明白"it"代指的对象。

#### 7. Why we use layer normalizaiton instead of batch normalization in Transformer ?

因为BN很依赖batch size，并且实际使用时需要计算且保存某一层神经网络batch的均值和方差等统计信息，而对于RNN来说每一个句子的长度不固定，深度不固定，不同的time-step需要保存不同的统计信息，所以BN不适用于RNN；
Layer normalization对同一层网络的输出做一个标准化，对样本本身进行操作，不受batch size影响，适用于RNN。

#### 8. Why we need position embedding in Transformer ?

transformer不像RNN根据time-step识别数据位置信息，它将整个句子都放进去，没有考虑到词序，进而丢失了位置信息，所以需要位置编码解决这个问题。

#### 9. Briefly describe what is self-attention and what is multi-head attention?

self-attention可以是一般attention的一种特殊情况，在self-attention中，Q=K=V每个序列中的单元和该序列中所有单元进行attention计算，它的特点在于无视词之间的距离直接计算依赖关系，能够学习一个句子的内部结构，并且可以并行计算。
Multi-head attention，Query，Key，Value首先进过一个线性变换，然后输入到放缩点积attention，注意这里要做h次，其实也就是所谓的多头，每一次算一个头。而且每次Q，K，V进行线性变换的参数W是不一样的。然后将h次的放缩点积attention结果进行拼接，再进行一次线性变换得到的值作为多头attention的结果，这样的好处是可以允许模型在不同的表示子空间里学习到相关的信息。

#### 10. What is the basic unit of GPT model?

GPT是单向语言模型，基于transformer中的Decoder。

#### 11. Briefly descibe how to use GPT in other NLP tasks?

classification：在文本前后加上起始和终止符号即可。
entailment：因输入有前提和假说两个句子，那就在两个句子中加入分隔符（delim）连接两条句子，作为输入，经过语言模型送入分类器。
similarity：因两条句子的顺序不影响结果，就按两种顺序分别放入语言模型得到各自的hidden state，将两种hidden state相加，送入分类器。
multiple choice：对于每个答案，都将context、问题、该答案以分隔符隔开连接起来，作为输入，经过语言模型送入分类器得到一个向量，将所有答案的向量送入softmax。

#### 12. What is masked language model in BERT ?

GPT中因为要完成语言模型的训练，也就要求Pre-Training预测下一个词的时候只能够看见当前以及之前的词，这也是GPT放弃原本Transformer的双向结构转而采用单向结构的原因。

BERT为了能够同时得到上下文的信息，而不是像GPT一样完全放弃下文信息，采用了双向的Transformer。但是这样一来，就无法再像GPT一样采用正常的语言模型来预训练了，因为BERT的结构导致每个Transformer的输出都可以看见整个句子的，无论你用这个输出去预测什么，都会“看见”参考答案，也就是“see itself”的问题。ELMo中虽然采用的是双向RNN，但是两个RNN之间是独立的，所以可以避免see itself的问题。

在Transformer中，我们即想要知道上文的信息，又想要知道下文的信息，但同时要保证整个模型不知道要预测词的信息，那么就干脆不要告诉模型这个词的信息就可以了。也就是说，BERT在输入的句子中，挖掉一些需要预测的词，然后通过上下文来分析句子，最终使用其相应位置的输出来预测被挖掉的词。这其实就像是在做完形填空 (Cloze)一样。
随机选择语料中 15% 的单词，把它抠掉，也就是用 [Mask] 掩码代替原始单词，然后要求模型去正确预测被抠掉的单词。但是这里有个问题：训练过程大量看到 [mask] 标记，但是真正后面用的时候是不会有这个标记的，这会引导模型认为输出是针对 [mask] 这个标记的，但是实际使用又见不到这个标记，这自然会有问题。

为了避免这个问题，Bert 改造了一下，15% 的被上天选中要执行 [mask] 替身这项光荣任务的单词中，只有 80% 真正被替换成 [mask] 标记，10% 被狸猫换太子随机替换成另外一个单词，10% 情况这个单词还待在原地不做改动。这就是 Masked 双向语音模型的具体做法。
这样一来就相当于告诉模型，我可能给你答案，也可能不给你答案，也可能给你错误的答案，有<MASK>的地方我会检查你的答案，没<MASK>的地方我也可能检查你的答案，所以<MASK>标签对你来说没有什么特殊意义，所以无论如何，你都要好好预测所有位置的输出。

#### 13. What are the inputs of BERT ?

BERT还提出了另外一种预训练方式NSP，与MLM同时进行，组成多任务预训练。这种预训练的方式就是往Transformer中输入连续的两个句子，左边的句子前面加上一个<CLS>标签，它的输出被用来判断两个句子之间是否是连续上下文关系。

它的输入部分是个线性序列，两个句子通过分隔符分割，最前面和最后增加两个标识符号。每个单词有三个 embedding：

position embedding，这是因为 NLP 中单词顺序是很重要的特征，需要在这里对位置信息进行编码；

word embedding, 每个单词的词向量；

segment embedding，因为训练数据都是由两个句子构成的，用来区分两个句子，每个句子有个句子整体的 embedding 项对应给每个单词，两个句子之间使用<SEP>标签予以区分。
把单词对应的三个 embedding 叠加，就形成了 Bert 的输入。

#### 14. Briely descibe how to use BERT in other NLP tasks.

对于句子关系类任务，和 GPT 类似，加上一个起始和终结符号，句子之间加个分隔符即可。对于输出来说，把第一个起始符号对应的 Transformer 最后一层位置上面串接一个 softmax 分类层即可；

对于分类问题，与 GPT 一样，只需要增加起始和终结符号，输出部分和句子关系判断任务类似改造；

对于序列标注问题，输入部分和单句分类是一样的，只需要输出部分 Transformer 最后一层每个单词对应位置都进行分类即可。

#### 15. What are the differences between these three models: GPT, BERT, GPT2.

GPT是基于transformer的decoder部分，是单向语言模型，采用了Pre-training + Fine-tuning的训练模式，使得大量无标记的数据得以利用。但是由于是单向语言模型，它在训练时只关注上文，无法结合下文词义训练；
BERT是基于transformer的encoder部分，是双向语言模型，它最关键两点，一点是特征抽取器采用 Transformer；第二点是预训练的时候采用双向语言模型。
GPT2相较于GPT引入更多参数，数据量更大，模型质量更高。